In [1]:
"""
To get the top3 category

Feature:
    - implement the sortby, a customized compare method
    - customize reduceByKey
"""

'\nTo get the top3 category\n\nFeature:\n    - implement the sortby, a customized compare method\n    - customize reduceByKey\n'

In [2]:
"""
first is to read the action table, then get the session_id_to_filter_rdd, then join, to get the filtered action table
"""

'\nfirst is to read the action table, then get the session_id_to_filter_rdd, then join, to get the filtered action table\n'

In [3]:
# for import
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession, HiveContext
from utils.spark_utils import insertHive, raw_data_to_filtered_full_info
from utils.data_model import category_action_count, top10_category
import uuid

In [4]:
sparkSession = (SparkSession
                    .builder
                    .appName('example-pyspark-read-and-write-from-hive')
                    .config("hive.metastore.uris", "thrift://localhost:9083", conf=SparkConf())
                    .enableHiveSupport()
                    .getOrCreate()
                    )

In [5]:
task_id = uuid.uuid1().__str__()
task_id

'af7859de-6cc8-11eb-b685-acde48001122'

In [6]:
sparkSession

In [7]:
# get the action table
user_action_rdd = sparkSession.sql("select * from user_visit_action_table").rdd

In [8]:
user_action_rdd.collect()

[Row(date='2021-01-27', user_id=69, session_id='a58034ae603b11ebb6eeacde48001122', page_id=70, action_time='2021-01-27 4:41:40', search_keyword=None, click_category_id=90, click_product_id=25, order_category_ids=None, order_product_ids=None, pay_category_ids=None, pay_product_ids=None, city_id=4),
 Row(date='2021-01-27', user_id=69, session_id='a58034ae603b11ebb6eeacde48001122', page_id=35, action_time='2021-01-27 4:09:59', search_keyword=None, click_category_id=-1, click_product_id=-1, order_category_ids=None, order_product_ids=None, pay_category_ids=78, pay_product_ids=88, city_id=2),
 Row(date='2021-01-27', user_id=69, session_id='a58034ae603b11ebb6eeacde48001122', page_id=29, action_time='2021-01-27 4:37:22', search_keyword=None, click_category_id=-1, click_product_id=-1, order_category_ids='1', order_product_ids='63', pay_category_ids=None, pay_product_ids=None, city_id=3),
 Row(date='2021-01-27', user_id=69, session_id='a58034ae603b11ebb6eeacde48001122', page_id=96, action_time='

In [9]:
# change to user_action -> (session_id, action)
session_to_action_rdd = user_action_rdd.map(lambda x : (x.session_id, x))

In [10]:
session_to_action_rdd.collect()

[('a58034ae603b11ebb6eeacde48001122',
  Row(date='2021-01-27', user_id=69, session_id='a58034ae603b11ebb6eeacde48001122', page_id=70, action_time='2021-01-27 4:41:40', search_keyword=None, click_category_id=90, click_product_id=25, order_category_ids=None, order_product_ids=None, pay_category_ids=None, pay_product_ids=None, city_id=4)),
 ('a58034ae603b11ebb6eeacde48001122',
  Row(date='2021-01-27', user_id=69, session_id='a58034ae603b11ebb6eeacde48001122', page_id=35, action_time='2021-01-27 4:09:59', search_keyword=None, click_category_id=-1, click_product_id=-1, order_category_ids=None, order_product_ids=None, pay_category_ids=78, pay_product_ids=88, city_id=2)),
 ('a58034ae603b11ebb6eeacde48001122',
  Row(date='2021-01-27', user_id=69, session_id='a58034ae603b11ebb6eeacde48001122', page_id=29, action_time='2021-01-27 4:37:22', search_keyword=None, click_category_id=-1, click_product_id=-1, order_category_ids='1', order_product_ids='63', pay_category_ids=None, pay_product_ids=None, c

In [11]:
# get the filtered session
filtered_sessionid_to_full_info = raw_data_to_filtered_full_info(sparkSession, user_action_rdd, sc)

In [12]:
filtered_sessionid_to_full_info.collect()

[('a57406b6603b11eb8e9facde48001122',
  'session_id=a57406b6603b11eb8e9facde48001122|search_keywords=Lamer,Lobsters,Facial Lotions,Vacuum,Mugs,Apple,Napkins,Machine Learning,Lenovo Laptops|click_categories=40,53,7,34,49,25,30,24,23,36,12,90,46,15,20,18,35|visit_length=3586|step_length=77|start_time=2021-01-27 13:00:07|age=32|professional=professional19|sex=Female|city=city28'),
 ('a5742178603b11eb8204acde48001122',
  'session_id=a5742178603b11eb8204acde48001122|search_keywords=Apple,Lenovo Laptops,Mugs,Facial Lotions,Vacuum|click_categories=51,95,34,48,7,80,88,19|visit_length=3234|step_length=33|start_time=2021-01-27 3:01:21|age=32|professional=professional19|sex=Female|city=city28'),
 ('a574fa80603b11ebb8bbacde48001122',
  'session_id=a574fa80603b11ebb8bbacde48001122|search_keywords=Facial Lotions,Lamer,Machine Learning,Vacuum,Mugs,Apple|click_categories=35,73,13,2,61,75,29,37,33,63|visit_length=3243|step_length=46|start_time=2021-01-27 1:00:04|age=32|professional=professional19|sex=F

In [13]:
# join
session_id_filtered_action_rdd = session_to_action_rdd.join(filtered_sessionid_to_full_info)

In [14]:
session_id_filtered_action_rdd.collect()

[('a553f092603b11eb997facde48001122',
  (Row(date='2021-01-27', user_id=44, session_id='a553f092603b11eb997facde48001122', page_id=74, action_time='2021-01-27 23:57:10', search_keyword=None, click_category_id=9, click_product_id=13, order_category_ids=None, order_product_ids=None, pay_category_ids=None, pay_product_ids=None, city_id=10),
   'session_id=a553f092603b11eb997facde48001122|search_keywords=Huawei Cell Phone,Lenovo Laptops,Napkins,Lamer,Machine Learning,Vacuum,Apple,Mugs,Facial Lotions|click_categories=9,95,62,8,19,0,33,22,39,49,97,67,76|visit_length=3579|step_length=66|start_time=2021-01-27 23:00:01|age=40|professional=professional89|sex=Female|city=city25')),
 ('a553f092603b11eb997facde48001122',
  (Row(date='2021-01-27', user_id=44, session_id='a553f092603b11eb997facde48001122', page_id=79, action_time='2021-01-27 23:24:03', search_keyword=None, click_category_id=-1, click_product_id=-1, order_category_ids='95', order_product_ids='17', pay_category_ids=None, pay_product_id

In [15]:
# pick out the session_id -> filtered action
def map_full_action_to_action(x):
    (session_id, (action, full_info)) = x
    return (session_id, action)
session_id_to_filtered_action_rdd = session_id_filtered_action_rdd.map(lambda x : map_full_action_to_action(x))

In [16]:
# this is the filtered session_id -> action
session_id_to_filtered_action_rdd.collect()

[('a553f092603b11eb997facde48001122',
  Row(date='2021-01-27', user_id=44, session_id='a553f092603b11eb997facde48001122', page_id=74, action_time='2021-01-27 23:57:10', search_keyword=None, click_category_id=9, click_product_id=13, order_category_ids=None, order_product_ids=None, pay_category_ids=None, pay_product_ids=None, city_id=10)),
 ('a553f092603b11eb997facde48001122',
  Row(date='2021-01-27', user_id=44, session_id='a553f092603b11eb997facde48001122', page_id=79, action_time='2021-01-27 23:24:03', search_keyword=None, click_category_id=-1, click_product_id=-1, order_category_ids='95', order_product_ids='17', pay_category_ids=None, pay_product_ids=None, city_id=1)),
 ('a553f092603b11eb997facde48001122',
  Row(date='2021-01-27', user_id=44, session_id='a553f092603b11eb997facde48001122', page_id=67, action_time='2021-01-27 23:38:15', search_keyword=None, click_category_id=95, click_product_id=36, order_category_ids=None, order_product_ids=None, pay_category_ids=None, pay_product_ids

In [17]:
# def get_top10_categories(sparksession, task_uuid, session_id_to_action_rdd):
#     """
#     what I want here is [('category1' : (click_num, order_num, pay_num))]
#     """
def action_to_count(x):
    action = x[1]
    id = ''
    
    click_count = 0
    order_count = 0
    pay_count = 0
    
    if action.click_category_id != -1:
        id = action.click_category_id
        click_count = 1
    elif action.order_category_ids != None:
        id = int(action.order_category_ids)
        order_count = 1
    elif action.pay_category_ids != None:
        id = action.pay_category_ids
        pay_count = 1
    
    return (id, (click_count, order_count, pay_count))
single_action_count_rdd = session_id_to_filtered_action_rdd.map(lambda x : action_to_count(x))

In [18]:
single_action_count_rdd.collect()

[(9, (1, 0, 0)),
 (95, (0, 1, 0)),
 (95, (1, 0, 0)),
 (62, (1, 0, 0)),
 ('', (0, 0, 0)),
 ('', (0, 0, 0)),
 ('', (0, 0, 0)),
 ('', (0, 0, 0)),
 (63, (0, 0, 1)),
 (7, (0, 0, 1)),
 (45, (0, 1, 0)),
 (76, (0, 1, 0)),
 ('', (0, 0, 0)),
 (35, (0, 1, 0)),
 (8, (1, 0, 0)),
 (19, (1, 0, 0)),
 (4, (0, 0, 1)),
 (24, (0, 0, 1)),
 (78, (0, 1, 0)),
 ('', (0, 0, 0)),
 (37, (0, 0, 1)),
 (27, (0, 0, 1)),
 (0, (1, 0, 0)),
 (87, (0, 1, 0)),
 (33, (1, 0, 0)),
 ('', (0, 0, 0)),
 (22, (1, 0, 0)),
 (100, (0, 0, 1)),
 (29, (0, 1, 0)),
 ('', (0, 0, 0)),
 (72, (0, 0, 1)),
 (8, (0, 0, 1)),
 (69, (0, 1, 0)),
 (59, (0, 0, 1)),
 (22, (0, 0, 1)),
 (39, (1, 0, 0)),
 ('', (0, 0, 0)),
 ('', (0, 0, 0)),
 ('', (0, 0, 0)),
 (80, (0, 0, 1)),
 (88, (0, 1, 0)),
 (8, (1, 0, 0)),
 (49, (1, 0, 0)),
 ('', (0, 0, 0)),
 (36, (0, 0, 1)),
 (15, (0, 0, 1)),
 (35, (0, 1, 0)),
 (63, (0, 1, 0)),
 (51, (0, 1, 0)),
 (49, (0, 1, 0)),
 (54, (0, 1, 0)),
 (97, (1, 0, 0)),
 (87, (0, 1, 0)),
 ('', (0, 0, 0)),
 (1, (1, 0, 0)),
 (23, (0, 1, 0)),

In [19]:
"""
search action, there is no category id
"""
def filter_search_action(x):
    return x[0] != ''
filtered_single_action_rdd = single_action_count_rdd.filter(lambda x : filter_search_action(x))

In [20]:
filtered_single_action_rdd.collect()

[(9, (1, 0, 0)),
 (95, (0, 1, 0)),
 (95, (1, 0, 0)),
 (62, (1, 0, 0)),
 (63, (0, 0, 1)),
 (7, (0, 0, 1)),
 (45, (0, 1, 0)),
 (76, (0, 1, 0)),
 (35, (0, 1, 0)),
 (8, (1, 0, 0)),
 (19, (1, 0, 0)),
 (4, (0, 0, 1)),
 (24, (0, 0, 1)),
 (78, (0, 1, 0)),
 (37, (0, 0, 1)),
 (27, (0, 0, 1)),
 (0, (1, 0, 0)),
 (87, (0, 1, 0)),
 (33, (1, 0, 0)),
 (22, (1, 0, 0)),
 (100, (0, 0, 1)),
 (29, (0, 1, 0)),
 (72, (0, 0, 1)),
 (8, (0, 0, 1)),
 (69, (0, 1, 0)),
 (59, (0, 0, 1)),
 (22, (0, 0, 1)),
 (39, (1, 0, 0)),
 (80, (0, 0, 1)),
 (88, (0, 1, 0)),
 (8, (1, 0, 0)),
 (49, (1, 0, 0)),
 (36, (0, 0, 1)),
 (15, (0, 0, 1)),
 (35, (0, 1, 0)),
 (63, (0, 1, 0)),
 (51, (0, 1, 0)),
 (49, (0, 1, 0)),
 (54, (0, 1, 0)),
 (97, (1, 0, 0)),
 (87, (0, 1, 0)),
 (1, (1, 0, 0)),
 (23, (0, 1, 0)),
 (73, (0, 0, 1)),
 (67, (1, 0, 0)),
 (28, (0, 1, 0)),
 (75, (0, 1, 0)),
 (76, (1, 0, 0)),
 (2, (1, 0, 0)),
 (23, (1, 0, 0)),
 (67, (0, 1, 0)),
 (23, (0, 1, 0)),
 (29, (0, 0, 1)),
 (22, (0, 0, 1)),
 (82, (0, 0, 1)),
 (21, (1, 0, 0)),


In [21]:
def action_count(a, b):
    return (a[0] + b[0], a[1] + b[1], a[2] + b[2])
filter_category_action_count_rdd = filtered_single_action_rdd.reduceByKey(action_count)

In [22]:
filter_category_action_count_rdd.collect()

[(0, (61, 71, 71)),
 (96, (65, 71, 66)),
 (48, (74, 79, 86)),
 (49, (69, 74, 70)),
 (97, (81, 60, 75)),
 (1, (75, 67, 58)),
 (2, (62, 74, 63)),
 (98, (67, 71, 69)),
 (50, (59, 71, 73)),
 (51, (70, 57, 67)),
 (3, (70, 64, 71)),
 (99, (70, 71, 87)),
 (4, (71, 65, 78)),
 (100, (67, 70, 73)),
 (52, (63, 72, 92)),
 (5, (83, 70, 73)),
 (53, (83, 63, 69)),
 (54, (67, 69, 68)),
 (6, (73, 72, 85)),
 (7, (68, 68, 79)),
 (55, (66, 70, 68)),
 (8, (74, 70, 78)),
 (56, (71, 72, 90)),
 (9, (74, 68, 83)),
 (57, (55, 75, 82)),
 (58, (74, 84, 82)),
 (10, (70, 53, 79)),
 (59, (65, 67, 72)),
 (11, (58, 57, 55)),
 (60, (75, 72, 70)),
 (12, (77, 66, 79)),
 (13, (55, 68, 73)),
 (61, (68, 69, 67)),
 (62, (58, 83, 54)),
 (14, (77, 83, 75)),
 (63, (67, 72, 77)),
 (15, (74, 83, 76)),
 (16, (65, 81, 74)),
 (64, (67, 74, 77)),
 (17, (74, 70, 65)),
 (65, (62, 71, 76)),
 (66, (61, 71, 58)),
 (18, (67, 62, 53)),
 (19, (77, 72, 82)),
 (67, (88, 69, 74)),
 (68, (76, 66, 62)),
 (20, (61, 73, 70)),
 (69, (62, 90, 76)),
 

In [23]:
"""
make it sortable
"""
def to_sortable_count(x):
    return (x[0], category_action_count(x[1][0], x[1][1], x[1][2]))
filtered_sortable_action_count_rdd = filter_category_action_count_rdd.map(lambda x : to_sortable_count(x))

In [24]:
filtered_sortable_action_count_rdd.collect()

[(0, <utils.data_model.category_action_count at 0x1318525c0>),
 (96, <utils.data_model.category_action_count at 0x131c66dd8>),
 (48, <utils.data_model.category_action_count at 0x131c666d8>),
 (49, <utils.data_model.category_action_count at 0x131c665f8>),
 (97, <utils.data_model.category_action_count at 0x131c66ef0>),
 (1, <utils.data_model.category_action_count at 0x131c66ac8>),
 (2, <utils.data_model.category_action_count at 0x131c66a90>),
 (98, <utils.data_model.category_action_count at 0x131c66c50>),
 (50, <utils.data_model.category_action_count at 0x131c66b70>),
 (51, <utils.data_model.category_action_count at 0x131c66be0>),
 (3, <utils.data_model.category_action_count at 0x131c66c18>),
 (99, <utils.data_model.category_action_count at 0x131c66940>),
 (4, <utils.data_model.category_action_count at 0x131c66a20>),
 (100, <utils.data_model.category_action_count at 0x131c66c88>),
 (52, <utils.data_model.category_action_count at 0x131c66f28>),
 (5, <utils.data_model.category_action_count

In [25]:
sorted_category_id_to_action_count_class_rdd = filtered_sortable_action_count_rdd.sortBy(lambda x : x[1], ascending=False)

In [26]:
sorted_category_id_to_action_count_class_rdd.collect()

[(28, <utils.data_model.category_action_count at 0x131f51978>),
 (67, <utils.data_model.category_action_count at 0x131f51f28>),
 (73, <utils.data_model.category_action_count at 0x131f51ba8>),
 (5, <utils.data_model.category_action_count at 0x131f51390>),
 (71, <utils.data_model.category_action_count at 0x131f51358>),
 (53, <utils.data_model.category_action_count at 0x131f519b0>),
 (44, <utils.data_model.category_action_count at 0x131f51da0>),
 (24, <utils.data_model.category_action_count at 0x131f4f8d0>),
 (97, <utils.data_model.category_action_count at 0x131f4f198>),
 (80, <utils.data_model.category_action_count at 0x131f4f080>),
 (38, <utils.data_model.category_action_count at 0x131f4fb00>),
 (94, <utils.data_model.category_action_count at 0x131f4fb70>),
 (31, <utils.data_model.category_action_count at 0x131f4ffd0>),
 (93, <utils.data_model.category_action_count at 0x131f4fbe0>),
 (14, <utils.data_model.category_action_count at 0x131f4fac8>),
 (19, <utils.data_model.category_action_c

In [27]:
def show_sort_class(x):
    return top10_category(task_id, x[0], x[1].click_count, x[1].order_count, x[1].pay_count)
sorted_category_id_to_action_count_rdd = sorted_category_id_to_action_count_class_rdd.map(lambda x : show_sort_class(x))

In [28]:
top10 = sorted_category_id_to_action_count_rdd.take(10)

In [30]:
top10_df = sparkSession.createDataFrame(top10)

In [32]:
insertHive(sparkSession, 'top10_categories', top10_df)

In [33]:
# check the result
sparkSession.sql("select * from top10_categories").rdd.collect()

[Row(task_id='af7859de-6cc8-11eb-b685-acde48001122', category_id=73, click_count=84, order_count=68, pay_count=53),
 Row(task_id='af7859de-6cc8-11eb-b685-acde48001122', category_id=67, click_count=88, order_count=69, pay_count=74),
 Row(task_id='af7859de-6cc8-11eb-b685-acde48001122', category_id=97, click_count=81, order_count=60, pay_count=75),
 Row(task_id='af7859de-6cc8-11eb-b685-acde48001122', category_id=80, click_count=80, order_count=78, pay_count=74),
 Row(task_id='af7859de-6cc8-11eb-b685-acde48001122', category_id=53, click_count=83, order_count=63, pay_count=69),
 Row(task_id='af7859de-6cc8-11eb-b685-acde48001122', category_id=24, click_count=81, order_count=66, pay_count=77),
 Row(task_id='af7859de-6cc8-11eb-b685-acde48001122', category_id=71, click_count=83, order_count=65, pay_count=67),
 Row(task_id='af7859de-6cc8-11eb-b685-acde48001122', category_id=5, click_count=83, order_count=70, pay_count=73),
 Row(task_id='af7859de-6cc8-11eb-b685-acde48001122', category_id=44, clic